<a href="https://colab.research.google.com/github/ThumbRocket/AI_Expert_Lecture_Files/blob/main/Lab2_2_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader

from torchvision.datasets import MNIST
from torchvision.transforms import Compose,ToTensor

from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
import PIL

#### Connect to local google drive & settings for export/import models

In [10]:
drive.mount('/content/gdrive')
!mkdir ./gdrive/'My Drive'/MNIST_models/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
mkdir: cannot create directory ‘./gdrive/My Drive/MNIST_models/’: File exists


#### DNN model

In [11]:
class MNISTDNN(nn.Module):
    def __init__(self,IMG_SIZE=28):
        super(MNISTDNN,self).__init__()
        self.fc1 = nn.Linear(IMG_SIZE*IMG_SIZE,32)
        self.BN1 = torch.nn.BatchNorm1d(32)
        self.fc2 = nn.Linear(32,10)

    def forward(self,x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.BN1(x)
        x = self.fc2(x)
        x = torch.softmax(x,dim=-1)
        return x

#### CNN model

In [12]:
class MNISTCNN(nn.Module):
    def __init__(self,IMG_SIZE=28):
        super(MNISTCNN,self).__init__()
        self.conv1 = nn.Conv2d(1,8,5,stride=2)
        self.BN1 = torch.nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8,8,5,stride=2)
        self.BN2 = torch.nn.BatchNorm2d(8)
        self.conv3 = nn.Conv2d(8,8,3,stride=1)
        self.fc = nn.Linear(8*2*2,10)

    def forward(self,x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.BN1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.BN2(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = x.view(-1,8*2*2)
        x = self.fc(x)
        x = torch.softmax(x,dim=-1)
        return x

#### Util function for calculating accuracy

In [13]:
def compute_acc(argmax,y):
    count = 0
    for i in range(len(argmax)):
        if argmax[i]==y[i]:
            count+=1
    return count / len(argmax)

#### hyperparameters & datasets

In [14]:
IMG_SIZE = 28
BATCH_SIZE = 256
LEARNING_RATE = 0.001
NUM_EPOCHES = 5

In [25]:
transforms = Compose([
    ToTensor(),
])

trainset = MNIST('/content/gdrive/My Drive/MNIST_models/',train=True,transform=transforms,download=True)
testset = MNIST('/content/gdrive/My Drive/MNIST_models/',train=False,transform=transforms,download=True)

args = {
    'num_workers' : 1,
    'batch_size' : BATCH_SIZE,
    'shuffle' : True,
}

train_loader = DataLoader(trainset,**args)
test_loader = DataLoader(testset,**args)

####Training part(DNN)

In [26]:
model = MNISTDNN(IMG_SIZE).cuda()

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
num_params = sum([np.prod(p.size()) for p in model_parameters])
print("number of parameters : {}".format(num_params))

optimizer = Adam(model.parameters(),lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(NUM_EPOCHES):
    tot_loss = 0.0

    for x,y in train_loader:
        optimizer.zero_grad()
        x = x.cuda().view(-1,IMG_SIZE*IMG_SIZE)
        y_ = model(x)
        loss = loss_fn(y_, y.cuda())
        loss.backward()
        tot_loss+=loss.item()
        optimizer.step()

    print("Epoch {}, Loss(train) : {}".format(epoch+1,tot_loss/BATCH_SIZE))
    if epoch % 2 == 1:
        x,y = next(iter(test_loader))
        x = x.cuda().view(-1,IMG_SIZE*IMG_SIZE)
        y_ = model(x)
        print(y_)
        _, argmax = torch.max(y_,dim=-1)
        test_acc = compute_acc(argmax,y.numpy())

        print("Acc(val) : {}".format(test_acc))

torch.save(model.state_dict(), "/content/gdrive/My Drive/MNIST_models/DNN.pt")

number of parameters : 25514
Epoch 1, Loss(train) : 1.598251092247665
Epoch 2, Loss(train) : 1.442443406675011
tensor([[1.4936e-03, 1.5286e-03, 4.1107e-04,  ..., 2.9324e-03, 1.4134e-04,
         5.6761e-04],
        [3.7576e-05, 2.8446e-06, 7.9534e-07,  ..., 2.0448e-04, 3.1563e-04,
         9.4248e-04],
        [6.3074e-04, 1.9177e-03, 2.2942e-02,  ..., 4.4542e-04, 1.9243e-04,
         1.6173e-03],
        ...,
        [1.2181e-03, 9.7542e-01, 2.9284e-04,  ..., 2.4005e-03, 5.0800e-03,
         1.5288e-03],
        [3.5470e-04, 9.9029e-01, 1.2189e-03,  ..., 2.5126e-03, 2.2706e-03,
         9.0730e-04],
        [9.1472e-05, 1.3824e-04, 9.9950e-01,  ..., 7.8784e-05, 1.5901e-05,
         3.6044e-07]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Acc(val) : 0.9453125
Epoch 3, Loss(train) : 1.4139149473048747
Epoch 4, Loss(train) : 1.400166512466967
tensor([[9.5460e-06, 2.0420e-06, 5.2733e-05,  ..., 3.5358e-04, 6.2252e-04,
         8.6219e-01],
        [3.0540e-07, 6.4194e-08, 7.1858e-06,  .

In [42]:
list(model.parameters())[3].requires_grad

True

In [49]:
list(model.parameters())[0].size()

torch.Size([32, 784])

In [50]:
np.prod(list(model.parameters())[0].size())

25088

In [34]:
# self.fc1(x) 784 * 32
# F.relu(x) 32
# self.BN1(x) 32
# self.fc2(x) 10 * 32
# torch.softmax(x,dim=-1) 10

for i in model.parameters():
  print(i.size())

torch.Size([32, 784])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([10, 32])
torch.Size([10])


In [17]:
model_test = MNISTDNN(IMG_SIZE).cuda()
model_test.load_state_dict(torch.load("/content/gdrive/My Drive/MNIST_models/DNN.pt"))
model_test.eval()
x,y = next(iter(test_loader))
x = x.cuda().view(-1,IMG_SIZE*IMG_SIZE)
y_ = model_test(x)
_, argmax = torch.max(y_,dim=-1)
test_acc = compute_acc(argmax,y.numpy())

print("Acc(test) : {}".format(test_acc))

Acc(test) : 0.953125


#### Training part(CNN)

In [19]:
model = MNISTCNN(IMG_SIZE).cuda()

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
num_params = sum([np.prod(p.size()) for p in model_parameters])
print("number of parameters : {}".format(num_params))

optimizer = Adam(model.parameters(),lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(NUM_EPOCHES):
    tot_loss = 0.0

    for x,y in train_loader:
        optimizer.zero_grad()
        x = x.cuda()
        y_ = model(x)
        loss = loss_fn(y_, y.cuda())
        loss.backward()
        tot_loss+=loss.item()
        optimizer.step()

    print("Epoch {}, Loss(train) : {}".format(epoch+1,tot_loss/BATCH_SIZE))
    if epoch % 2 == 1:
        model.eval()

        x,y = next(iter(test_loader))
        x = x.cuda()
        y_ = model(x)
        _, argmax = torch.max(y_,dim=-1)
        test_acc = compute_acc(argmax,y.numpy())

        print("Acc(test) : {}".format(test_acc))

        model.train()

torch.save(model.state_dict(), "/content/gdrive/My Drive/MNIST_models/CNN.pt")

number of parameters : 2762
Epoch 1, Loss(train) : 1.6816038410179317
Epoch 2, Loss(train) : 1.4096929337829351
Acc(test) : 0.97265625
Epoch 3, Loss(train) : 1.3866860098205507
Epoch 4, Loss(train) : 1.3779380801133811
Acc(test) : 0.97265625
Epoch 5, Loss(train) : 1.3725390094332397


In [ ]:
model_test = MNISTCNN(IMG_SIZE).cuda()
model_test.load_state_dict(torch.load("/content/gdrive/My Drive/MNIST_models/CNN.pt"))
model_test.eval()
x,y = next(iter(test_loader))
x = x.cuda()
y_ = model_test(x)
_, argmax = torch.max(y_,dim=-1)
test_acc = compute_acc(argmax,y.numpy())

print("Acc(test) : {}".format(test_acc))